# 실습 세션 1: LLM 및 프롬프트 엔지니어링 활용

본 세션에서는 Azure AI Foundry를 사용하여 LLM을 배포하고, Chat Playground와 Jupyter Notebook에서 다양한 프롬프트 엔지니어링 기법을 실습합니다.

## 1. Azure AI Foundry에서 GPT-4o mini 모델 배포하기

Azure AI Foundry(이전의 Azure OpenAI Service)를 통해 최신 GPT-4o mini 모델을 배포할 수 있습니다. 

- Azure Portal에서 [AI Foundry 리소스]를 생성합니다.
- "모델 배포" 메뉴에서 `gpt-4o mini`(예: gpt-4o-2024-04-09) 모델을 선택하여 배포합니다.
- 배포 이름(예: gpt-4o-mini-dev)과 엔드포인트, API 키를 확인합니다.
- 배포가 완료되면, 해당 엔드포인트와 키를 활용해 다양한 애플리케이션에서 사용할 수 있습니다.

> **참고:** GPT-4o mini는 최신 고성능 LLM으로, 빠른 응답 속도와 저렴한 비용이 특징입니다. Azure AI Foundry에서 제공하는 다양한 모델 중 하나로, 실시간 대화, 요약, 생성 등 다양한 작업에 적합합니다.

## 2. Azure OpenAI 연결 설정

먼저 필요한 패키지를 설치하고 Azure OpenAI 서비스에 연결하기 위한 설정을 진행합니다.

In [1]:
# 필요한 라이브러리 설치
!pip install -r requirements.txt

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

# .env 파일에서 환경 변수 로드 (없는 경우 직접 입력)
load_dotenv()

# Azure OpenAI 설정
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

# AzureOpenAI 클라이언트 생성
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version=api_version
)

# 모델 배포 이름
# gpt-4o mini 배포 시 아래와 같이 이름을 지정할 수 있습니다.
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini-dev") # 배포된 모델 이름

## 3. Azure AI Foundry에서 LLM 모델 배포 확인

Azure AI Foundry(이전의 Azure OpenAI Service)에서 배포한 LLM 모델 정보를 확인해봅시다.

In [3]:
# 배포된 모델 목록 확인
try:
    # 참고: 현재 버전의 AzureOpenAI 클라이언트에서는 
    # 모든 배포 목록을 직접 확인하는 기능이 제한적일 수 있습니다.
    # 아래 코드는 예시일 뿐이며, 실제 환경에 따라 다를 수 있습니다.
    print("배포된 모델 정보:")
    print(f"- 현재 사용 중인 모델: {deployment_name}")
    print("Azure Portal에서 배포된 모든 모델을 확인할 수 있습니다.")

    # 배포된 모델 테스트 (간단한 테스트)
    test_response = client.chat.completions.create(
        model=deployment_name,
        messages=[{"role": "user", "content": "안녕하세요"}],
        max_tokens=10
    )
    print(f"\n모델 응답 테스트 성공: {test_response.choices[0].message.content}")
except Exception as e:
    print(f"모델 확인 중 오류 발생: {e}")
    print("Azure Portal에서 직접 배포 상태를 확인해주세요.")

배포된 모델 정보:
- 현재 사용 중인 모델: gpt-4o-mini
Azure Portal에서 배포된 모든 모델을 확인할 수 있습니다.

모델 응답 테스트 성공: 안녕하세요! 어떻게 도와드릴까요?


## 4. 기본 프롬프트 엔지니어링

프롬프트 엔지니어링의 기본 기법들을 실습해봅시다.

In [4]:

def get_completion(prompt, model=deployment_name):
    """지정된 프롬프트와 모델을 사용하여 Azure OpenAI API에 요청을 보내고 응답을 반환합니다."""
    # 메시지 구성
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    # 채팅 완성 요청
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=4096
    )
    
    # 응답에서 메시지 내용 추출
    return response.choices[0].message.content

# 간단한 프롬프트 테스트
simple_prompt = "Azure란 무엇인가요?"
print(get_completion(simple_prompt))

Azure는 Microsoft가 제공하는 클라우드 컴퓨팅 플랫폼입니다. Azure는 다양한 클라우드 서비스와 솔루션을 제공하여 애플리케이션을 구축, 배포 및 관리할 수 있도록 지원합니다. 이 플랫폼은 다음과 같은 여러 기능을 제공합니다:

1. **인프라 서비스(IaaS)**: 가상 머신, 네트워크, 스토리지와 같은 기본 인프라를 제공하여 사용자에게 필요한 리소스를 쉽게 관리할 수 있게 합니다.

2. **플랫폼 서비스(PaaS)**: 개발자가 애플리케이션을 구축하고 배포하는 데 필요한 환경을 제공하며, 데이터베이스, 애플리케이션 호스팅, 개발 도구 등을 포함합니다.

3. **소프트웨어 서비스(SaaS)**: 사전 구축된 애플리케이션을 클라우드에서 제공하여 사용자가 별도의 설치나 관리 없이 이용할 수 있도록 합니다.

4. **데이터 분석 및 AI**: Azure는 데이터 분석, 머신러닝, 인공지능 관련 서비스도 제공하여 기업이 데이터를 활용하여 인사이트를 얻고 의사결정을 내릴 수 있도록 지원합니다.

5. **IoT 서비스**: 사물인터넷(IoT) 솔루션을 구축할 수 있는 도구와 서비스를 제공하여 다양한 디바이스를 연결하고 데이터를 수집 및 분석할 수 있게 합니다.

6. **보안 및 관리 도구**: Azure는 클라우드 환경에서의 보안을 강화하고 리소스를 효과적으로 관리할 수 있는 다양한 도구와 서비스를 제공합니다.

Azure는 전 세계에 있는 데이터 센터를 통해 높은 가용성과 확장성을 제공하며, 다양한 산업 분야와 비즈니스 요구에 맞춘 솔루션을 지원합니다.


### 4.1 명확한 지시사항 제공하기

LLM에게 명확한 지시사항을 제공하면 더 좋은 결과를 얻을 수 있습니다.

In [5]:
# 명확하지 않은 지시사항
unclear_prompt = "클라우드 컴퓨팅에 대해 알려줘."
print("\n=== 명확하지 않은 지시사항 ===\n")
print(get_completion(unclear_prompt))

# 명확한 지시사항
clear_prompt = """클라우드 컴퓨팅에 대해 다음 형식으로 설명해줘:
1. 정의 (50단어 이내)
2. 주요 이점 (5가지 나열)
3. 주요 서비스 제공업체 (3개)
4. Azure의 주요 서비스 (5가지)
"""
print("\n=== 명확한 지시사항 ===\n")
print(get_completion(clear_prompt))


=== 명확하지 않은 지시사항 ===

클라우드 컴퓨팅(Cloud Computing)은 인터넷 상의 서버를 통해 데이터 저장, 관리, 처리 등의 컴퓨팅 자원을 제공하는 서비스 모델입니다. 이를 통해 사용자는 물리적인 서버나 인프라를 소유하거나 관리할 필요 없이, 필요한 만큼의 컴퓨팅 자원을 쉽게 이용할 수 있습니다.

클라우드 컴퓨팅의 주요 특징은 다음과 같습니다:

1. **온디맨드 서비스(On-Demand Self-Service)**: 사용자는 필요할 때 원하는 만큼의 자원을 즉시 요청하고 사용할 수 있습니다.

2. **광범위한 네트워크 접근성(Broad Network Access)**: 클라우드 서비스는 인터넷을 통해 다양한 장치에서 접근할 수 있습니다.

3. **자원 공유(Resource Pooling)**: 여러 사용자가 동일한 물리적 자원을 공유하며, 자원 할당이 동적으로 이루어집니다.

4. **신속한 유연성(Rapid Elasticity)**: 필요에 따라 자원의 확장 및 축소가 용이합니다.

5. **측정된 서비스(Measured Service)**: 사용자는 실제 사용량에 따라 요금을 지불하며, 서비스 제공자는 자원 사용을 모니터링하고 관리합니다.

클라우드 컴퓨팅은 주로 세 가지 서비스 모델로 나눌 수 있습니다:

1. **인프라 서비스(IaaS, Infrastructure as a Service)**: 가상화된 컴퓨팅 자원(서버, 스토리지, 네트워크 등)을 제공하는 서비스입니다. 예: Amazon EC2, Google Compute Engine.

2. **플랫폼 서비스(PaaS, Platform as a Service)**: 애플리케이션 개발 및 배포를 위한 플랫폼을 제공하는 서비스입니다. 예: Google App Engine, Microsoft Azure.

3. **소프트웨어 서비스(SaaS, Software as a Service)**: 소프트웨어 애플리케이션을 인터넷을 통해 제공하는 서비스입니다. 예: Google Works

### 4.2. 역할 부여하기 (Role Prompting)

LLM에게 특정 역할을 부여하면 해당 관점에서 응답을 생성할 수 있습니다.

In [6]:
# 역할 없는 프롬프트
no_role_prompt = "Azure Kubernetes Service(AKS)의 장점을 설명해줘."
print("\n=== 역할 없는 프롬프트 ===\n")
print(get_completion(no_role_prompt))

# 역할 부여 프롬프트
role_prompt = """당신은 5년 경력의 클라우드 아키텍트입니다. 
기업의 IT 관리자에게 Azure Kubernetes Service(AKS)를 도입해야 하는 이유와 장점을 
기술적 측면과 비즈니스 측면에서 설득력 있게 설명해주세요."""
print("\n=== 역할 부여 프롬프트 ===\n")
print(get_completion(role_prompt))


=== 역할 없는 프롬프트 ===

Azure Kubernetes Service (AKS)는 Microsoft Azure에서 제공하는 관리형 Kubernetes 서비스로, 여러 가지 장점을 가지고 있습니다. 아래는 AKS의 주요 장점들입니다.

1. **관리 용이성**: AKS는 Kubernetes 클러스터의 설정, 관리 및 유지보수를 자동화하여 사용자가 인프라에 대한 고민 없이 애플리케이션 개발에 집중할 수 있도록 도와줍니다.

2. **자동 확장**: AKS는 수요에 따라 클러스터를 자동으로 확장할 수 있는 기능을 제공하여, 트래픽이 증가할 때 리소스를 자동으로 추가하고, 감소할 때는 리소스를 줄일 수 있습니다.

3. **Azure 통합**: AKS는 Azure의 다양한 서비스와 통합되어, Azure Active Directory, Azure Monitor, Azure Policy 등과 쉽게 연동할 수 있습니다. 이를 통해 보안, 모니터링, 관리 측면에서 많은 이점을 제공합니다.

4. **비용 효율성**: AKS는 관리형 서비스로, Kubernetes 마스터 노드에 대한 비용이 발생하지 않습니다. 사용자는 필요한 만큼의 워커 노드만 비용을 지불하면 됩니다.

5. **보안**: Azure의 보안 기능을 활용하여 클러스터를 보호할 수 있습니다. Azure Active Directory와의 통합을 통해 사용자 인증 및 권한 부여를 관리할 수 있으며, 네트워크 보안 그룹과 같은 Azure 보안 기능을 활용할 수 있습니다.

6. **CI/CD 통합**: Azure DevOps, GitHub Actions 등의 CI/CD 툴과 쉽게 통합할 수 있어, 애플리케이션 배포 및 업데이트 프로세스를 자동화할 수 있습니다.

7. **다양한 운영 체제 및 언어 지원**: AKS는 다양한 운영 체제와 프로그래밍 언어를 지원하여, 개발자들이 선호하는 기술 스택을 이용할 수 있도록 유연성을 제공합니다.

8. **커뮤니티 지원**: Kubernetes는 오픈 소스 프로젝트로

### 4.3. 단계별 사고 유도 (Chain-of-Thought)

복잡한 문제를 해결할 때는 LLM에게 단계별로 생각하도록 유도하는 것이 효과적입니다.

In [7]:
# 직접적인 질문
direct_prompt = "Azure에서 3-tier 웹 애플리케이션을 구축하는 방법을 알려줘."
print("\n=== 직접적인 질문 ===\n")
print(get_completion(direct_prompt))

# 단계별 사고 유도 프롬프트
cot_prompt = """Azure에서 3-tier 웹 애플리케이션을 구축하려고 합니다. 
다음 단계에 따라 설계 방법을 설명해주세요:

1. 먼저, 각 계층(프레젠테이션, 비즈니스 로직, 데이터)에 적합한 Azure 서비스를 선택하세요.
2. 다음으로, 이러한 서비스들이 어떻게 연결되는지 설명하세요.
3. 그 다음, 보안 측면에서 고려해야 할 사항을 설명하세요.
4. 마지막으로, 확장성을 위한 설계 방법을 제안하세요.

각 단계별로 구분하여 자세히 설명해주세요."""
print("\n=== 단계별 사고 유도 프롬프트 ===\n")
print(get_completion(cot_prompt))


=== 직접적인 질문 ===

Azure에서 3-tier 웹 애플리케이션을 구축하는 과정은 다음과 같습니다. 3-tier 아키텍처는 일반적으로 프레젠테이션 계층, 비즈니스 로직 계층, 데이터 계층으로 나뉘어져 있습니다.

### 1. Azure 리소스 그룹 생성
먼저 Azure에서 리소스 그룹을 생성하여 모든 리소스를 관리합니다.

1. Azure 포털에 로그인합니다.
2. "리소스 그룹"을 선택하고 "추가"를 클릭합니다.
3. 리소스 그룹의 이름과 지역을 설정한 후 "검토 + 만들기"를 클릭하여 리소스 그룹을 생성합니다.

### 2. 데이터 계층 (Database)
Azure SQL Database 또는 Cosmos DB를 사용하여 데이터 계층을 설정합니다.

1. Azure 포털에서 "SQL 데이터베이스" 또는 "Cosmos DB"를 선택합니다.
2. "추가" 버튼을 클릭합니다.
3. 데이터베이스의 이름, 서버, 가격 책정 계층 등을 설정합니다.
4. 데이터베이스가 생성되면 필요한 테이블과 스키마를 만들 수 있습니다.

### 3. 비즈니스 로직 계층 (Backend)
Azure App Service를 이용하여 비즈니스 로직을 구현합니다.

1. Azure 포털에서 "App Services"를 선택합니다.
2. "추가" 버튼을 클릭합니다.
3. 웹 애플리케이션의 이름, 구독, 리소스 그룹, 런타임 스택(.NET, Node.js 등)을 선택합니다.
4. 애플리케이션이 배포되면 필요한 API 및 비즈니스 로직을 개발하여 배포합니다.

### 4. 프레젠테이션 계층 (Frontend)
Azure Static Web Apps 또는 Azure App Service를 사용하여 프론트엔드 애플리케이션을 호스팅합니다.

1. Azure 포털에서 "Static Web Apps" 또는 "App Services"를 선택합니다.
2. "추가" 버튼을 클릭합니다.
3. 프론트엔드 애플리케이션의 이름, 구독, 리소스 그룹 등을 설정합니다.
4. GitHub와 연동하거나 로컬에서

### 4.4. Few-Shot 학습

몇 가지 예시를 제공하여 LLM이 패턴을 학습하도록 유도할 수 있습니다.

In [8]:
# Few-shot 프롬프트
few_shot_prompt = """다음은 클라우드 서비스에 대한 간결한 설명입니다:

서비스: Azure Virtual Machine
설명: 다양한 크기와 운영체제를 지원하는 확장 가능한 컴퓨팅 리소스로, IaaS 솔루션을 제공합니다.

서비스: AWS Lambda
설명: 서버리스 컴퓨팅 서비스로, 코드 실행만 관리하며 인프라 관리가 필요 없는 이벤트 기반 애플리케이션을 구축할 수 있습니다.

서비스: Google Cloud Storage
설명: 확장 가능하고 내구성 있는 객체 스토리지 솔루션으로, 정적 콘텐츠 호스팅부터 빅데이터 분석까지 다양한 용도로 활용됩니다.

위의 형식을 따라 아래 서비스에 대한 설명을 작성해주세요:

서비스: Azure Kubernetes Service (AKS)
설명:

서비스: Azure Functions
설명:

서비스: Azure Cosmos DB
설명:
"""
print(get_completion(few_shot_prompt))

서비스: Azure Kubernetes Service (AKS)  
설명: Kubernetes의 관리형 서비스로, 컨테이너화된 애플리케이션을 손쉽게 배포, 관리 및 확장할 수 있는 플랫폼을 제공합니다.

서비스: Azure Functions  
설명: 이벤트 기반 서버리스 컴퓨팅 서비스로, 코드를 실행하는 데 필요한 인프라를 자동으로 관리하며, 다양한 트리거를 통해 손쉽게 애플리케이션 로직을 구현할 수 있습니다.

서비스: Azure Cosmos DB  
설명: 글로벌 분산형 NoSQL 데이터베이스 서비스로, 다양한 데이터 모델을 지원하며 자동 스케일링, 고가용성 및 낮은 지연시간을 제공하여 애플리케이션의 성능을 최적화합니다.


## 5. 고급 프롬프트 엔지니어링 기법

보다 복잡한 작업을 수행하기 위한 고급 프롬프트 엔지니어링 기법을 살펴봅시다.

### 5.1. 시스템 메시지 활용하기

시스템 메시지를 사용하여 LLM의 전반적인 행동을 정의할 수 있습니다.

In [9]:
def get_completion_with_system(system_message, user_message, model=deployment_name):
    """시스템 메시지와 사용자 메시지를 사용하여 Azure OpenAI API에 요청을 보내고 응답을 반환합니다."""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=800
    )
    return response.choices[0].message.content

# 기술 전문가 시스템 메시지
tech_system_message = """당신은 Microsoft Azure 전문가입니다. 항상 기술적으로 정확한 정보를 제공하며, 
복잡한 개념도 초보자가 이해할 수 있도록 쉽게 설명합니다. 가능한 경우 코드 예제나 구체적인 단계를 포함해주세요."""

user_question = "Azure Functions와 Azure Logic Apps의 차이점은 무엇인가요?"

print("=== 시스템 메시지 활용 ===\n")
print(get_completion_with_system(tech_system_message, user_question))

=== 시스템 메시지 활용 ===

Azure Functions와 Azure Logic Apps는 모두 Azure에서 제공하는 서버리스 컴퓨팅 솔루션이지만, 사용 목적과 기능에서 차이가 있습니다. 아래에 두 서비스의 주요 차이점을 설명하겠습니다.

### Azure Functions

1. **정의**: Azure Functions는 이벤트 기반의 서버리스 컴퓨팅 서비스로, 특정 이벤트가 발생했을 때 코드를 실행하는 데 사용됩니다. 주로 API, 웹훅, 스케줄러 등 다양한 이벤트에 반응합니다.

2. **코드 작성**: Azure Functions는 C#, JavaScript, Python 등 여러 프로그래밍 언어로 코드를 작성할 수 있습니다. 개발자가 직접 로직을 구현할 수 있습니다.

3. **유연성**: 복잡한 비즈니스 로직을 구현할 수 있으며, 다양한 외부 서비스와 상호작용할 수 있습니다. 예를 들어, 데이터베이스에 연결하거나 다른 API를 호출하는 것이 가능합니다.

4. **비용 모델**: 사용한 만큼만 요금이 부과되는 'Consumption Plan'과 같은 다양한 요금제를 제공합니다.

### Azure Logic Apps

1. **정의**: Azure Logic Apps는 다양한 서비스와 앱을 연결하여 워크플로우를 자동화하는 데 사용되는 서비스입니다. 주로 비즈니스 프로세스 자동화에 적합합니다.

2. **시각적 디자인**: Logic Apps는 GUI 기반의 디자이너를 제공하여 사용자가 복잡한 코드를 작성하지 않고도 워크플로우를 구성할 수 있습니다. 다양한 커넥터를 통해 서비스 간의 연결이 용이합니다.

3. **사전 정의된 연결**: Microsoft와 서드파티 서비스에 대한 많은 커넥터가 제공되어, 이메일 전송, 데이터베이스 업데이트, 파일 저장 등 다양한 작업을 쉽게 수행할 수 있습니다.

4. **비용 모델**: Logic Apps는 실행한 트리거와 액션에 따라 요금이 부과됩니다.

### 요약

- **Azure Functions*

### 5.2. 다중 단계 지시사항 (Multi-Step Instructions)

복잡한 작업을 여러 단계로 나누어 수행하도록 지시할 수 있습니다.

In [10]:
multi_step_prompt = """Azure Web App을 사용하여 간단한 웹 애플리케이션을 배포하는 과정을 알려주세요.
다음 순서로 설명해주세요:

1. 필요한 사전 요구 사항 나열
2. Azure Portal에서 Web App 리소스 생성 단계
3. VS Code를 사용하여 로컬에서 웹 앱 개발하기
4. Azure에 웹 앱 배포하기
5. 배포 후 모니터링 및 문제 해결 방법

각 단계마다 구체적인 명령어나 설정 옵션을 포함해주세요."""

print("=== 다중 단계 지시사항 ===\n")
print(get_completion(multi_step_prompt))

=== 다중 단계 지시사항 ===

Azure Web App을 사용하여 간단한 웹 애플리케이션을 배포하는 과정에 대해 설명하겠습니다. 다음 단계에 따라 진행하세요.

### 1. 필요한 사전 요구 사항 나열
- **Azure 계정**: Azure Portal에 접근할 수 있는 계정이 필요합니다. [Azure 무료 계정 만들기](https://azure.microsoft.com/free/)를 통해 무료로 계정을 생성할 수 있습니다.
- **Visual Studio Code (VS Code)**: 웹 애플리케이션 개발을 위한 소스 코드 편집기입니다. [VS Code 다운로드](https://code.visualstudio.com/)에서 설치할 수 있습니다.
- **Node.js (선택 사항)**: Node.js를 사용하는 경우 [Node.js 다운로드](https://nodejs.org/)에서 설치합니다.
- **Azure CLI**: Azure CLI를 사용하여 명령줄에서 Azure 리소스를 관리할 수 있습니다. [Azure CLI 설치](https://docs.microsoft.com/cli/azure/install-azure-cli)에서 설치합니다.
- **Git**: 버전 관리를 위해 Git을 사용할 수 있습니다. [Git 다운로드](https://git-scm.com/downloads)에서 설치합니다.

### 2. Azure Portal에서 Web App 리소스 생성 단계
1. Azure Portal에 로그인합니다.
2. 왼쪽 메뉴에서 **"리소스 만들기"**를 클릭합니다.
3. **"Web App"**을 검색하여 선택합니다.
4. **Web App 만들기** 페이지에서 다음 정보를 입력합니다:
   - **구독**: 사용할 구독 선택
   - **리소스 그룹**: 새 리소스 그룹을 만들거나 기존 리소스 그룹 선택
   - **앱 이름**: 고유한 웹 앱 이름 입력
   - **배포 방법**: 코드 또는 Docker 컨테이너 선택 (예: 코드)
   -

### 5.3. 출력 형식 지정 (Output Formatting)

LLM의 응답을 특정 형식으로 받을 수 있습니다.

In [11]:
format_prompt = """Azure의 주요 AI 서비스 5가지를 JSON 형식으로 제공해주세요. 각 서비스에 대해 다음 정보를 포함해주세요:
- 서비스 이름
- 간단한 설명
- 주요 기능 (3가지)
- 일반적인 사용 사례 (2가지)
- 관련 Azure 서비스 (2가지)

응답은 다음과 같은 JSON 형식이어야 합니다:
{
  "ai_services": [
    {
      "name": "서비스 이름",
      "description": "설명",
      "key_features": ["기능1", "기능2", "기능3"],
      "use_cases": ["사용 사례1", "사용 사례2"],
      "related_services": ["관련 서비스1", "관련 서비스2"]
    },
    ...
  ]
}"""

json_response = get_completion(format_prompt)
print("=== 출력 형식 지정 (JSON) ===\n")
print(json_response)

# JSON 응답을 파싱하여 구조화된 데이터로 사용 가능
try:
    parsed_response = json.loads(json_response)
    print("\n=== 첫 번째 서비스 정보 ===\n")
    print(f"서비스 이름: {parsed_response['ai_services'][0]['name']}")
    print(f"설명: {parsed_response['ai_services'][0]['description']}")
    print("주요 기능:")
    for feature in parsed_response['ai_services'][0]['key_features']:
        print(f"- {feature}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 오류: {e}")

=== 출력 형식 지정 (JSON) ===

```json
{
  "ai_services": [
    {
      "name": "Azure Cognitive Services",
      "description": "다양한 AI 기능을 제공하는 API 및 서비스 모음으로, 이미지, 텍스트, 음성 및 비디오 분석을 지원합니다.",
      "key_features": ["자연어 처리", "이미지 인식", "음성 인식"],
      "use_cases": ["챗봇 개발", "감정 분석"],
      "related_services": ["Azure Bot Services", "Azure Machine Learning"]
    },
    {
      "name": "Azure Machine Learning",
      "description": "데이터 과학자와 개발자가 머신러닝 모델을 구축, 학습 및 배포할 수 있도록 지원하는 플랫폼입니다.",
      "key_features": ["자동 머신러닝", "모델 관리", "데이터 준비 도구"],
      "use_cases": ["예측 분석", "추천 시스템"],
      "related_services": ["Azure Databricks", "Azure DevOps"]
    },
    {
      "name": "Azure Bot Services",
      "description": "대화형 AI 봇을 쉽게 만들고 배포할 수 있도록 지원하는 서비스입니다.",
      "key_features": ["다양한 채널 지원", "언어 이해 (LUIS)", "봇 프레임워크"],
      "use_cases": ["고객 지원 챗봇", "정보 제공 봇"],
      "related_services": ["Azure Cognitive Services", "Azure Functions"]
    },
    {
      "name": "Azure Form Recognizer",
      

## 6. 실전 프롬프트 엔지니어링

실제 업무에서 활용할 수 있는 프롬프트 엔지니어링 예제를 살펴봅시다.

### 6.1. Azure 아키텍처 설계 조언 받기

In [12]:
architecture_prompt = """당신은 수석 Azure 솔루션 아키텍트입니다. 다음 요구사항에 맞는 Azure 아키텍처를 설계해주세요:

요구사항:
- 고가용성 웹 애플리케이션 (ASP.NET Core)
- 실시간 데이터 처리 (IoT 디바이스에서 수집된 센서 데이터)
- 대용량 데이터 저장 및 분석
- AI 기반 예측 모델을 통한 이상 징후 감지
- 철저한 보안 및 규정 준수 (개인정보 포함)
- 비용 효율적인 설계

다음 사항을 포함해주세요:
1. 아키텍처 다이어그램 (텍스트로 표현)
2. 각 Azure 서비스의 선택 이유
3. 확장성 및 장애 조치 전략
4. 예상 비용 최적화 방안
5. 보안 고려사항"""

print("=== Azure 아키텍처 설계 조언 ===\n")
print(get_completion(architecture_prompt))

=== Azure 아키텍처 설계 조언 ===

아래는 요구사항에 맞춘 Azure 아키텍처 설계입니다.

### 1. 아키텍처 다이어그램 (텍스트로 표현)

```
[IoT Device] 
    |
    v
[Azure IoT Hub]
    |
    v
[Azure Stream Analytics]
    |
    v
[Azure Data Lake Storage]
    |
    v
[Azure Databricks (for data processing & AI model)]
    |
    v
[Power BI / Azure Synapse Analytics (for analysis)]
    |
    v
[ASP.NET Core Web App (Azure App Service)]
    |
    v
[Azure SQL Database / Cosmos DB (for transactional data)]
    |
    v
[Azure Key Vault (for secrets management)]
```

### 2. 각 Azure 서비스의 선택 이유
- **Azure IoT Hub**: IoT 디바이스에서 데이터 수집 및 관리, 보안 연결을 제공.
- **Azure Stream Analytics**: 실시간 데이터 스트리밍 및 이벤트 처리, 센서 데이터의 즉각적인 분석 가능.
- **Azure Data Lake Storage**: 대용량의 비정형 데이터 저장, 확장성 및 비용 효율성 제공.
- **Azure Databricks**: 데이터 처리 및 AI 모델 훈련을 위한 통합 분석 환경, Apache Spark 기반의 데이터 처리 능력.
- **Power BI / Azure Synapse Analytics**: 데이터 시각화 및 분석을 위한 BI 도구, 대규모 데이터 분석 지원.
- **ASP.NET Core Web App (Azure App Service)**: 고가용성 및 자동 확장을 지원하는 웹 애플리케이션 플랫폼.
- **Azure SQL

### 6.2. Azure CLI 명령어 생성

In [13]:
cli_prompt = """Azure CLI 명령어를 사용하여 다음 작업을 수행하는 스크립트를 생성해주세요:

1. 리소스 그룹 생성 (이름: rg-workshop-prod, 위치: Korea Central)
2. Azure Kubernetes Service(AKS) 클러스터 생성 (3개 노드, 시스템 할당 관리 ID 사용)
3. Azure Container Registry(ACR) 생성 및 AKS와 연결
4. Azure Key Vault 생성 및 필요한 비밀 저장 (DB 연결 문자열, API 키)
5. Azure Monitor를 사용하여 AKS 클러스터 모니터링 설정

각 명령어에 대한 간단한 설명을 주석으로 추가해주세요."""

print("=== Azure CLI 명령어 생성 ===\n")
print(get_completion(cli_prompt))

=== Azure CLI 명령어 생성 ===

아래는 Azure CLI를 사용하여 요청하신 작업을 수행하는 스크립트입니다. 각 명령어에 대한 설명은 주석으로 추가되어 있습니다.

```bash
#!/bin/bash

# 1. 리소스 그룹 생성 (이름: rg-workshop-prod, 위치: Korea Central)
az group create --name rg-workshop-prod --location "Korea Central"

# 2. Azure Kubernetes Service(AKS) 클러스터 생성 (3개 노드, 시스템 할당 관리 ID 사용)
az aks create \
    --resource-group rg-workshop-prod \
    --name aks-workshop \
    --node-count 3 \
    --enable-managed-identity \
    --generate-ssh-keys

# 3. Azure Container Registry(ACR) 생성 및 AKS와 연결
# ACR 이름은 고유해야 하므로, 알파벳 소문자와 숫자만 포함된 이름을 설정합니다.
acr_name="acrworkshopprod$(date +%s)"  # ACR 이름 생성
az acr create --resource-group rg-workshop-prod --name $acr_name --sku Basic

# ACR와 AKS 연결
az aks update --resource-group rg-workshop-prod --name aks-workshop --attach-acr $acr_name

# 4. Azure Key Vault 생성 및 필요한 비밀 저장 (DB 연결 문자열, API 키)
# Key Vault 이름은 고유해야 하므로, 알파벳 소문자와 숫자만 포함된 이름을 설정합니다.
kv_name="kvworkshopprod$(date +%s)"  # Key Vault 이름 생성
az keyvault create --name $kv_n

## 7. 실습: 나만의 프롬프트 작성하기

이제 지금까지 배운 내용을 바탕으로 나만의 프롬프트를 작성해봅시다.

In [14]:
# 여기에 직접 프롬프트를 작성하고 실행해보세요
my_prompt = """
# 여기에 프롬프트를 작성하세요
"""

# 주석을 제거하고 실행하세요
# print(get_completion(my_prompt))

## 8. 마무리 및 다음 단계

이 세션에서는 LLM과 프롬프트 엔지니어링의 기본 및 고급 기법을 학습했습니다. 다음 세션에서는 GitHub Copilot과 AutoGen을 활용하여 Multi AI Agent 앱을 구현해볼 예정입니다.

### 학습한 내용 요약
- Azure AI Foundry에서 LLM 모델 배포 및 사용 방법
- 기본 프롬프트 엔지니어링 기법 (명확한 지시사항, 역할 부여, 단계별 사고 유도, Few-Shot 학습)
- 고급 프롬프트 엔지니어링 기법 (시스템 메시지, 다중 단계 지시사항, 출력 형식 지정)
- 실전 프롬프트 엔지니어링 사례

### 추가 학습 자료
- [Azure OpenAI Service 공식 문서](https://docs.microsoft.com/azure/cognitive-services/openai/)
- [Azure AI Foundry 공식 문서](https://docs.microsoft.com/azure/ai-services/)